In [1]:
import pandas as pd 
import numpy as np
import statistics

In [44]:
sess = pd.read_csv("/Users/alex/Desktop/KABAOOM/ubc_vc_2019_sessions.csv", nrows=10000) 
users = pd.read_csv("/Users/alex/Desktop/KABAOOM/ubc_vc_2019_users.csv") 

In [45]:
sess.drop(['Unnamed: 0', 'session_id'], axis=1, inplace=True)
users.drop('Unnamed: 0', axis=1, inplace=True)
sess = sess.fillna(0)
users = users.fillna(0)
sess.head(3)

,uid,connectivity,platform,in_alliance,session_start_time,end_time,install_session,hc_store_visits,tutorial_completion,tutorial_complete_timestamp,...,other_wins,pve_fights,pve_wins,pvp_fights,pvp_wins,total_fights,total_fight_time,transactions,spend,level
0,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,ON_WIFI,ios,True,2018-02-25 10:03:44+00:00,2018-02-25 11:21:19+00:00,False,0,0,0,...,0.0,71.0,67.0,0.0,0.0,71.0,1804.0,0,0.0,27.0
1,5M+BWt53rH5e9bFlDca5ho95CU+diI+A7lgstgMXchA=,ON_WIFI,ios,False,2018-02-28 10:21:28+00:00,2018-02-28 11:28:20+00:00,False,0,0,0,...,0.0,35.0,26.0,0.0,0.0,35.0,1738.0,0,0.0,27.0
2,qSOL52erIrvrHWneB5+vKRhFgNjCrizD0l822AwSDUU=,ON_WIFI,android,True,2018-02-23 14:40:50+00:00,2018-02-23 16:54:46+00:00,False,0,0,0,...,0.0,75.0,54.0,0.0,0.0,75.0,2187.0,0,0.0,27.0


In [46]:
#all users are unique
print(len(users))
users.uid.nunique()

296248


296248

In [47]:
#connectivity is a boolean var (It can only be wifi or mobile), so here I basically encoded Wifi as 1 (and mobile = 0) to replace "connectivity"
sess = pd.concat([sess, pd.get_dummies(sess.connectivity)], axis=1)
sess.drop(['ON_CELL', 'connectivity'], axis=1, inplace=True)

#let's do the same for platform (make a separate android/ios variable; ios = 1, android = 0)
sess = pd.concat([sess, pd.get_dummies(sess.platform)], axis=1)
sess1 = sess.drop(['android', 'platform'], axis=1)[:]

sess1.head()

,uid,in_alliance,session_start_time,end_time,install_session,hc_store_visits,tutorial_completion,tutorial_complete_timestamp,authentication_types,sc_spent,...,pve_wins,pvp_fights,pvp_wins,total_fights,total_fight_time,transactions,spend,level,ON_WIFI,ios
0,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-25 10:03:44+00:00,2018-02-25 11:21:19+00:00,False,0,0,0,kabam,200.0,...,67.0,0.0,0.0,71.0,1804.0,0,0.0,27.0,1,1
1,5M+BWt53rH5e9bFlDca5ho95CU+diI+A7lgstgMXchA=,False,2018-02-28 10:21:28+00:00,2018-02-28 11:28:20+00:00,False,0,0,0,kabam,0.0,...,26.0,0.0,0.0,35.0,1738.0,0,0.0,27.0,1,1
2,qSOL52erIrvrHWneB5+vKRhFgNjCrizD0l822AwSDUU=,True,2018-02-23 14:40:50+00:00,2018-02-23 16:54:46+00:00,False,0,0,0,device,32394.0,...,54.0,0.0,0.0,75.0,2187.0,0,0.0,27.0,1,0
3,2XgqjlslNWlK6tnD2ijQ25hlq7LX25VL/OFMZiw83Uk=,True,2018-02-10 03:14:45+00:00,2018-02-10 04:32:03+00:00,False,1,0,0,kabam,24185.0,...,20.0,8.0,6.0,38.0,1192.0,0,0.0,26.0,0,0
4,xhnSNrtb0IrQUm7wx5fdw3v3XiM7wIN5bMoA/xy2cu8=,True,2018-03-03 10:45:34+00:00,2018-03-03 17:11:12+00:00,False,1,0,0,kabam,7500.0,...,132.0,120.0,102.0,258.0,7013.0,0,0.0,32.0,1,1


In [48]:
#Let's create a new feature to determine how long each session is, based on the start/end times
sess1.end_time = pd.to_datetime(sess1.end_time)
sess1.session_start_time = pd.to_datetime(sess1.session_start_time)
sess1['session_length'] = (sess1.end_time - sess1.session_start_time).astype('timedelta64[s]') #take difference and convert to seconds 
print(sess1['session_length'][:3])

#it would also be interesting to see what percentage of time they spend "fighting". Here we're assuming fight time is in seconds
sess1['fight_ratio'] = sess1.total_fight_time/sess1.session_length
print(sess1.fight_ratio[:3])

0    4655.0
1    4012.0
2    8036.0
Name: session_length, dtype: float64
0    0.38754
1    0.43320
2    0.27215
Name: fight_ratio, dtype: float64


In [49]:
#just to get an idea of what kind of money people are spending in the game
print(sess1.sort_values(['spend'], ascending=0)[:10].filter(['spend', 'transactions', 'hc_store_visits', 'sc_spent']))
print(sess1.sort_values(['transactions'], ascending=0)[:10].filter(['spend', 'transactions', 'hc_store_visits', 'sc_spent']))
print(sess1.sort_values(['hc_store_visits'], ascending=0)[:10].filter(['spend', 'transactions', 'hc_store_visits', 'sc_spent']))

#create new feature, amount spent per transaction
sess1['spent_per_transaction'] = sess1.spend/sess1.transactions if [sess.transactions>0] else 0

#some summary stats here:
print('')
print('Summary stats:')

print(statistics.mean(sess1[sess1.tutorial_completion==1].session_length), '= mean time to complete the tutorial (s)')

print(statistics.mean(sess1.transactions), '            = avg # of transactions per person:')

print(len(sess1[sess1.transactions>0])/len(sess1), '             = % of people that make a transaction')

print(statistics.mean(sess1[sess1.transactions>0].spent_per_transaction), '= avg amt spent per transaction')

            spend  transactions  hc_store_visits  sc_spent
1749  3054.937398             2                3  114274.0
3394  2926.162968             1                1     800.0
9860  2909.351863             1                6   58751.0
8435  2907.930605             1                4  173392.0
2129  2907.360813             1                3   53412.0
2083  2906.788812             1                1   17973.0
8420  2848.672063             1                4    8611.0
2951  1601.401277             6               14   99049.0
222   1476.899521             1                2       0.0
897   1424.192606             1                1   44644.0
            spend  transactions  hc_store_visits  sc_spent
2951  1601.401277             6               14   99049.0
370   1195.130859             4                2   83801.0
2447   995.993287             4                7   53390.0
2206   597.134830             4                4  301324.0
245    397.429713             3                3   50561

In [10]:
#see if there's any variation in spending between authentication types. We'll encode this as dummies later when users are joined with sessions
print(pd.get_dummies(sess.authentication_types, prefix='auth').columns)
sesst=pd.concat([sess, pd.get_dummies(sess.authentication_types, prefix='auth')], axis=1)
print(sesst.columns)
sesst['auth_google'] = sesst['auth_0']*sess.spend
statistics.mean(sesst['auth_0'])

#not significant. But I'll leave it here, maybe you'll get some other ideas to try

Index(['auth_0', 'auth_device', 'auth_gamecenter', 'auth_google',
       'auth_kabam'],
      dtype='object')
Index(['uid', 'platform', 'in_alliance', 'session_start_time', 'end_time',
       'install_session', 'hc_store_visits', 'tutorial_completion',
       'tutorial_complete_timestamp', 'authentication_types', 'sc_spent',
       'sc_earned', 'xp_earned', 'ava_fights', 'ava_wins', 'ave_fights',
       'ave_wins', 'other_fights', 'other_wins', 'pve_fights', 'pve_wins',
       'pvp_fights', 'pvp_wins', 'total_fights', 'total_fight_time',
       'transactions', 'spend', 'level', 'ON_WIFI', 'android', 'ios',
       'session_length', 'fight_ratio', 'spent_per_transaction', 'auth_0',
       'auth_device', 'auth_gamecenter', 'auth_google', 'auth_kabam'],
      dtype='object')


0.0002

In [11]:
users.head()

,uid,in_alliance,device_model,device_os,registered_email,num_friends,power,geo,language,level,total_logins,platform,install_time,first_spend_time,last_login_time,last_spend_time,lt_spend,tutorial_complete,played_For
0,iAVWIbMaJHm20cPrLxgABE3tbzMOlrFCthS+vk07h40=,True,SM-J730FM,Android OS 7.0,True,0,384.0,BY,ru,17.0,30,android,2018-02-09 20:08:51+00:00,0,2018-02-19 09:22:13+00:00,0,0.0,True,825202.0
1,czL3c1UsK3BRtPbSckCby0ohFqyS7IbTChSUiD5vaGc=,False,Studio C HD,Android OS 6.0,False,0,117.0,TT,en,1.0,1,android,2018-02-02 21:39:16+00:00,0,2018-02-02 21:39:16+00:00,0,0.0,False,0.0
2,eyz3+noW7V1AiLQ1Ddkt3jtigmC9IRSHea/gyir2Yo0=,False,SM-J730F,Android OS 7.0,False,0,161.0,IL,it,9.0,24,android,2018-02-09 11:56:29+00:00,0,2018-02-15 11:51:33+00:00,0,0.0,True,518104.0
3,7AbWa2/OIu4ac3+JA86d4J3RhWCsVbhuHtNjjER0scE=,False,SM-J730F,Android OS 7.0,False,0,571.0,BG,ru,25.0,68,android,2018-02-04 13:02:53+00:00,0,2018-02-21 15:19:17+00:00,0,0.0,True,1476984.0
4,XpyOIGDpHelCdpM96veWlu2qgGa4g5OwheuB0v8Tw5E=,False,SM-J500H,Android OS 6.0.1,True,0,118.0,HU,en,1.0,457,android,2018-02-24 15:16:53+00:00,0,2019-02-16 09:26:42+00:00,0,0.0,True,30823789.0


In [12]:
#how many people playing with each language
np.unique(users.language, return_counts=True)

(array(['ar', 'de', 'en', 'es', 'fr', 'id', 'it', 'ja', 'ko', 'nl', 'no',
        'pt', 'ru', 'th', 'tr', 'zh-CN', 'zh-TW'], dtype=object),
 array([  4617,   6316, 123607,  41355,   8481,   2502,   5307,   2392,
          1996,   1434,    116,  25819,  29422,   2875,  11167,  25914,
          2928], dtype=int64))

In [51]:
#total time played for users
users.install_time = pd.to_datetime(users.install_time)
users.last_login_time = pd.to_datetime(users.last_login_time)
users['played_for'] = (users.last_login_time - users.install_time).astype('timedelta64[s]') #take difference and convert to seconds 
print(users['played_for'][:5])

0      825202.0
1           0.0
2      518104.0
3     1476984.0
4    30823789.0
Name: played_for, dtype: float64


In [52]:
#our top-spenders
users.sort_values(['lt_spend'], ascending=0)[:5]

,uid,in_alliance,device_model,device_os,registered_email,num_friends,power,geo,language,level,total_logins,platform,install_time,first_spend_time,last_login_time,last_spend_time,lt_spend,tutorial_complete,played_for
81899,lVim7dAs6KD4u6S+dCzIB+jsUKN+3br2pxSoHESKhLQ=,True,"iPhone10,5",iOS 12.1.2,True,31,10333.0,CN,zh-CN,60.0,5561,iphone,2018-02-06 04:04:03+00:00,2018-02-09 05:50:58+00:00,2019-02-26 07:48:45+00:00,2019-02-24 13:12:43+00:00,2476028.6,True,33277482.0
11997,R0rKeIza57XGsdFhjATfoBJgS27yzb7x0mjZn9Vy6SE=,True,"iPhone10,4",iOS 11.4.1,True,10,9594.0,AT,de,60.0,3098,iphone,2018-02-08 22:26:35+00:00,2018-02-27 18:36:04+00:00,2019-02-26 07:54:07+00:00,2019-02-22 20:38:22+00:00,1664490.3,True,33038852.0
272371,Iu8uWS+7qaAS0hYIXnJfru9Kdx0/FcLomP9MuEv1qu8=,False,"iPhone10,5",iOS 12.1,True,2,5150.0,US,en,53.0,1586,iphone,2018-02-17 23:22:03+00:00,2018-02-18 16:26:32+00:00,2018-11-12 18:12:35+00:00,2018-10-07 16:53:28+00:00,1380493.6,True,23136632.0
28458,m4hAwQzuDwbQweikAFkKAb+KKpEYZaLc4fMDymz5ydw=,True,"iPad7,3",iOS 12.1,True,41,7388.0,QA,en,60.0,3395,iphone,2018-02-20 13:37:49+00:00,2018-05-17 22:47:35+00:00,2019-02-26 07:38:38+00:00,2019-02-24 18:56:42+00:00,1087822.8,True,32032849.0
284446,ehIa9KTSM0NPnGsA2vDLbqAWDd3fJi1amcDTFi3lGkQ=,True,SM-T710,Android OS 7.0,True,11,7881.0,US,en,60.0,3492,android,2018-02-18 10:35:04+00:00,2018-02-24 23:45:39+00:00,2019-02-26 08:58:27+00:00,2019-02-26 00:03:48+00:00,868178.2,True,32221403.0


In [53]:
print(users.uid.nunique())
print(sess1.uid.nunique())

296248
2381


In [54]:
#match each session with an ID (jointype is inner so a session-less user will not appear)
joined = pd.merge(sess1,users,on='uid', validate='many_to_one')
joined.head(5)

,uid,in_alliance_x,session_start_time,end_time,install_session,hc_store_visits,tutorial_completion,tutorial_complete_timestamp,authentication_types,sc_spent,...,level_y,total_logins,platform,install_time,first_spend_time,last_login_time,last_spend_time,lt_spend,tutorial_complete,played_for
0,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-25 10:03:44+00:00,2018-02-25 11:21:19+00:00,False,0,0,0,kabam,200.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,0,2018-03-03 17:52:00+00:00,0,0.0,True,1212242.0
1,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-24 10:14:34+00:00,2018-02-24 11:19:50+00:00,False,1,0,0,kabam,15196.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,0,2018-03-03 17:52:00+00:00,0,0.0,True,1212242.0
2,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-25 13:52:57+00:00,2018-02-25 13:54:44+00:00,False,0,0,0,kabam,0.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,0,2018-03-03 17:52:00+00:00,0,0.0,True,1212242.0
3,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-25 09:39:45+00:00,2018-02-25 09:48:40+00:00,False,0,0,0,kabam,0.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,0,2018-03-03 17:52:00+00:00,0,0.0,True,1212242.0
4,ufGXdwBvXjlK90ywe5O4zmsjlD5pBexlkLErQkUMnvg=,True,2018-02-24 14:57:40+00:00,2018-02-24 15:02:18+00:00,False,0,0,0,kabam,0.0,...,31.0,214,iphone,2018-02-17 17:07:58+00:00,0,2018-03-03 17:52:00+00:00,0,0.0,True,1212242.0


In [55]:
print(joined.shape)
print(joined.columns)

(10000, 50)
Index(['uid', 'in_alliance_x', 'session_start_time', 'end_time',
       'install_session', 'hc_store_visits', 'tutorial_completion',
       'tutorial_complete_timestamp', 'authentication_types', 'sc_spent',
       'sc_earned', 'xp_earned', 'ava_fights', 'ava_wins', 'ave_fights',
       'ave_wins', 'other_fights', 'other_wins', 'pve_fights', 'pve_wins',
       'pvp_fights', 'pvp_wins', 'total_fights', 'total_fight_time',
       'transactions', 'spend', 'level_x', 'ON_WIFI', 'ios', 'session_length',
       'fight_ratio', 'spent_per_transaction', 'in_alliance_y', 'device_model',
       'device_os', 'registered_email', 'num_friends', 'power', 'geo',
       'language', 'level_y', 'total_logins', 'platform', 'install_time',
       'first_spend_time', 'last_login_time', 'last_spend_time', 'lt_spend',
       'tutorial_complete', 'played_for'],
      dtype='object')
